# Base Case

In [18]:
%load_ext autoreload
%autoreload 2
# The next line is to ensure inline plots in this notebook
%matplotlib inline

import powergama
import powergama.scenarios as pgs
import powergama.plots as pgplots
import pandas as pd
import numpy as np
import pathlib
import folium
import time

year = 2020
datapath_GridData = pathlib.Path().parent/f"data/nordic/data_{year}"
file_storval_filling = pathlib.Path("data//nordic/data_storagevalues/profiles_storval_filling.csv")
file_30y_profiles = pathlib.Path("data/nordic/data_timeseries/timeseries_profiles.csv.zip")
# datapath_ProfileData = pathlib.Path().parent/f"data/nordic"
# timerange = range(0, 24)
date_start = "2020-01-01"
date_end = "2020-02-01"
sql_file = f"powergama_{year}.sqlite"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Read Data

In [19]:
data = powergama.GridData()
data.readGridData(nodes=datapath_GridData / "node.csv",
                  ac_branches=datapath_GridData / "branch.csv",
                  dc_branches=datapath_GridData / "dcbranch.csv",
                  generators=datapath_GridData / "generator.csv",
                  consumers=datapath_GridData / "consumer.csv")

profiles_30y = pd.read_csv(file_30y_profiles, index_col=0, parse_dates=True)
profiles_30y["const"] = 1
data.profiles = profiles_30y[(profiles_30y.index >= date_start) & (profiles_30y.index < date_end)].reset_index()
data.storagevalue_time = data.profiles[["const"]]
storval_filling = pd.read_csv(file_storval_filling)
data.storagevalue_filling = storval_filling
data.timerange = list(range(data.profiles.shape[0]))
data.timeDelta = 1.0  # hourly data

num_hours = data.timerange[-1] - data.timerange[0]
print(f'Simulation hours: {num_hours}')
num_years = num_hours / (365.2425 * 24)
print(f'Simulation years: {np.round(num_years, 3)}')

# Filter offshore wind farms by year:
data.generator = data.generator[~(data.generator["year"] > year)].reset_index(drop=True)

# remove zero capacity generators:
m_gen_hascap = data.generator["pmax"] > 0
data.generator = data.generator[m_gen_hascap].reset_index(drop=True)


Simulation hours: 743
Simulation years: 0.085


### Plot Map    

In [20]:
f = folium.Figure(width=700, height=800)
m=pgplots.plotMap(data,spread_nodes_r=0.05)
m.fit_bounds([(data.node["lat"].min(),data.node["lon"].min()),(data.node["lat"].max(),data.node["lon"].max())]) 
m.add_to(f)

### Solve Base Case

In [21]:
lp = powergama.LpProblem(data)
res = powergama.Results(data, sql_file, replace=True)
start_time = time.time()
lp.solve(res, solver="gurobi")
end_time = time.time()
print("\nSimulation time = {:.2f} seconds".format(end_time - start_time))
print("System cost", sum(res.getSystemCost(timeMaxMin=[4, 6]).values()))
print("Mean area price", sum(res.getAreaPricesAverage(timeMaxMin=[4, 6]).values()) / len(res.getAreaPricesAverage()))


Computing B and DA matrices...
Initialising LP problem...
Found synchronous area (size = 4), using ref node = DK1 0
Found synchronous area (size = 58), using ref node = DK2 0
Found synchronous area (size = 1), using ref node = DK_hub
Found synchronous area (size = 1), using ref node = GB
Found synchronous area (size = 1), using ref node = NL
Replacing existing SQLite file "C:\Users\aleks\Python_\TET4510\powergama\scenarios\powergama_2020.sqlite"
:) Using direct python interface to solver
Solving...
1% 2% 3% 4% 5% 6% 7% 8% 9% 10% 11% 12% 13% 14% 15% 16% 17% 18% 19% 20% 21% 22% 23% 24% 25% 26% 27% 28% 29% 30% 31% 32% 33% 34% 35% 36% 37% 38% 39% 40% 41% 42% 43% 44% 45% 46% 47% 48% 49% 50% 51% 52% 53% 54% 55% 56% 57% 58% 59% 60% 61% 62% 63% 64% 65% 66% 67% 68% 69% 70% 71% 72% 73% 74% 75% 76% 77% 78% 79% 80% 81% 82% 83% 84% 85% 86% 87% 88% 89% 90% 91% 92% 93% 94% 95% 96% 97% 98% 99% 100% 
Simulation time = 53.82 seconds
System cost 2857940.386276195
Mean area price 24.157227310185483


In [22]:
# res.plotEnergyMix(relative=False)
# res.getEnergyBalanceInArea(area="NO", spillageGen="wind_offshore")
